In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [38]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [39]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.19
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,1,0,0,1,0,0,0,0,0,1.60
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.49
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.36
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.98


In [40]:
df_test.head()

,Id,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,1,c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
1,2,[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
2,3,[nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,4,[nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
4,5,c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [41]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [42]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


**EDA**

**Interesting question:**
- Which feature occures most often? / Which feature occured least often
- are there features that occure oftentimes together?
- are there features, which never go together? or which always go together?
- How do features correlate

In [43]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)


'\nExample Feature Engineering\n\nthis calculates the length of each smile string and adds a feature column with those lengths\nNote: this is NOT a good feature and will result in a lower score!\n'

In [44]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape

Train features: (1000000, 256)
Train gap: (1000000,)
Test features: (824230, 256)


In [45]:
#Y_train_original --> holds GAP Values in the Train set from the source file
Y_train_original = Y_train

#X_train_original --> features (258) of the Train set from the source file
X_train_original = X_train

#X_test_original --> features (258) of the Test set from the source file
X_test_original = X_test

In [46]:
from sklearn.cross_validation import train_test_split
#Do a Train/Test split based on Train Data X and Y
X_train, X_test, Y_train, Y_test = train_test_split(X_train_original, Y_train_original, test_size=0.3, random_state=0)

**Linear Regression**

In [74]:
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_test)

In [75]:
print("Explained variance score: 1 is perfect prediction: %0.2f") % (LR.score(X_test, Y_test))
rmse_LR = sqrt(mean_squared_error(Y_test, LR_pred))
print ("RMSE: %0.2f") % rmse_LR

Explained variance score: 1 is perfect prediction: 0.46
RMSE: 0.30


**Random Forest**

In [79]:
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)

In [81]:
print("Explained variance score: 1 is perfect prediction: %0.2f") % (RF.score(X_test, Y_test))
rmse_RF = sqrt(mean_squared_error(Y_test, RF_pred))
print ("RMSE: %0.2f") % rmse_RF

Explained variance score: 1 is perfect prediction: 0.55
RMSE: 0.27


In [51]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [52]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)

**Ridge Regression**

In [59]:
#Conduct a Ridge Regression with 5-fold cross validation
from sklearn.linear_model import RidgeCV 
LR_ridge = RidgeCV(alphas=(0.001, 0.01, 0.1, 1.0, 10.0, 100.0), cv=5)
LR_ridge.fit(X_train, Y_train)
LR_ridge_pred = LR_ridge.predict(X_test)

In [83]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse_ridge = sqrt(mean_squared_error(Y_test, LR_ridge_pred))

print ("Explained variance score: 1 is perfect prediction: %0.2f") % LR_ridge.score(X_test, Y_test)

print ("RMSE: %0.2f") % rmse_ridge

Explained variance score: 1 is perfect prediction: 0.46
RMSE: 0.30


In [ ]:
#...

In [ ]:
"""from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

# Fit estimators
ESTIMATORS = {
    "Extra trees": ExtraTreesRegressor(n_estimators=10, max_features=32,
                                       random_state=0),
    "K-nn": KNeighborsRegressor(),
    "Linear regression": LinearRegression(),
    "Ridge": RidgeCV(),
}"""

In [ ]:
"""y_test_predict = dict()
for name, estimator in ESTIMATORS.items():
    estimator.fit(X_train, Y_train)
    y_test_predict[name] = estimator.predict(X_test)
"""

**Approach**

1. Split X_train and Y_train into seperate train/test samples of 0.7/0.3 ration
2. Conduct the following regression models:
    - Linear Regression
    - Ridge Regression
    - Lasso Regression
    - Elastic Net

3. Compare the models
4. (Optional) Try a Bayesian Approach



**Open Questions**
1. Which Scoring Parameter should we use to evaluate each Method? Always the same, like R^2 or others?

